In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
!pip install tf2cv tensorflow>=2.0.0

In [0]:
from tensorflow.python.client import device_lib
dev=device_lib.list_local_devices()
dev[len(dev)-1]

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12848766024359734846
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"

In [0]:
#global consts
IMG_SIZE='137x236'
HEIGHT = 137
WIDTH = 236
BATCH_SIZE=128
DROP_OUT=0.5
FOLD=0
TRAIN_DIR = './train/'
TRAIN_IMG_DIR=TRAIN_DIR+str(IMG_SIZE)
EPOCHS = 200
STATS_FS = (0.055029564364430086, 0.17228061284674265)
STATS_128 = (0.0692, 0.2051)
EXPERIMENT='sernext_3cmc_fullsize_wod3'
INPUT_DIR ='/content/drive/My Drive/kaggle/bengali/input/'
OUTPUT_DIR = '/content/drive/My Drive/kaggle/bengali/output/'
IMPORT_DIR = '/content/drive/My Drive/kaggle/bengali/nbs/'
EXPERIMENT_DIR = OUTPUT_DIR + EXPERIMENT+'/'
CHKP_DIR = EXPERIMENT_DIR+'chkp/'
OUTPUT_SUBDIR = ''
DATASET_224='224x224-bengali.zip'
DATASET_128='grapheme-imgs-128x128.zip'
DATASET_FS = 'bengali-full-size.zip'
TRAIN_WITH_FOLD = 'train_with_fold_20.csv'
PRETRAINED = False
PRETRAINED_WEIGHTS = ''
LOG_FILE = EXPERIMENT_DIR+'log_'+EXPERIMENT+'.csv'
#INITIAL_EPOCH=40
#HEAD_WD=1e-3
#BACKBONE_WD = 1e-4

if IMG_SIZE == 128: DATASET=DATASET_128
if IMG_SIZE == 224: DATASET=DATASET_224
if IMG_SIZE == '137x236' : DATASET=DATASET_FS


#lr sheduler consts
LR_MAX = 0.001
LR_MIN = 0.00001
T_MAX = 100

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append(IMPORT_DIR+'py/models/')
sys.path.append(IMPORT_DIR+'py/utils/')
sys.path.append(IMPORT_DIR+'py/callbacks/')
sys.path.append(IMPORT_DIR+'py/opts/')

In [0]:
#import local modules
#from mish import Mish,mish
from generators import *
from kaggle_metric import *
#from cosine import CosineAnnealingScheduler
from kagglevalidation import KaggleValidationFS
#from lookahead import Lookahead
from se_resnext50 import build_se_resnext50_model_fs
#from md121v2 import build_md121_v2_model
#from losses import categorical_focal_loss


#import external modules
import numpy as np 
import pandas as pd 

import os
import errno

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight

In [0]:
try:
  os.mkdir(TRAIN_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(TRAIN_DIR+' already exists')
    else:
        raise
try:
  os.mkdir(TRAIN_IMG_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(TRAIN_IMG_DIR+' already exists')
    else:
        raise

try:
  os.mkdir(EXPERIMENT_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(EXPERIMENT_DIR+' already exists')
    else:
        raise

try:
  os.mkdir(CHKP_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(CHKP_DIR+' already exists')
    else:
        raise


os.system('cp '+ '"'+INPUT_DIR+DATASET+'" ' + TRAIN_DIR)
os.system('cp '+ '"'+INPUT_DIR+TRAIN_WITH_FOLD+'" ' + TRAIN_DIR)

/content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/ already exists
/content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ already exists


0

In [0]:
if PRETRAINED: os.system('cp '+ '"'+OUTPUT_DIR+OUTPUT_SUBDIR+PRETRAINED_WEIGHTS+'" '+TRAIN_DIR)

In [0]:
os.system('unzip -q '+TRAIN_DIR+DATASET+ ' -d '+ TRAIN_IMG_DIR)

0

In [0]:
!ls train

137x236  bengali-full-size.zip	train_with_fold_20.csv


In [0]:
dataset_np = pd.read_csv(TRAIN_DIR+TRAIN_WITH_FOLD).to_numpy()
dataset_np[:,0]+='.png'
dataset_m = dataset_np.shape[0]


valid_m = np.where(dataset_np[:,6]==FOLD)[0].shape[0]
train_m = dataset_m-valid_m

fold_train_inds = np.where(dataset_np[:,6] != FOLD)
fold_valid_inds = np.where(dataset_np[:,6] == FOLD)

train_np = dataset_np[fold_train_inds]
valid_np = dataset_np[fold_valid_inds]

train_df = pd.DataFrame(train_np)
valid_df = pd.DataFrame(valid_np)

train_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)
valid_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)

train_df.drop(columns=[5,6], inplace=True)
valid_df.drop(columns=[5,6], inplace=True)

In [0]:
cons_unique=np.unique(train_np[:,3])
cons_y_train = train_np[:,3]
vowel_unique=np.unique(train_np[:,2])
vowel_y_train = train_np[:,2]
root_unique =np.unique(train_np[:,1])
root_y_train = train_np[:,1]

cons_class_weights = class_weight.compute_class_weight('balanced',
                                                 cons_unique,
                                                 cons_y_train)

vowel_class_weights = class_weight.compute_class_weight('balanced',
                                                 vowel_unique,
                                                 vowel_y_train)

root_class_weights = class_weight.compute_class_weight('balanced',
                                                 root_unique,
                                                 root_y_train)
cons_cw_dict=dict(enumerate(cons_class_weights))
vowel_cw_dict=dict(enumerate(vowel_class_weights))
root_cw_dict=dict(enumerate(root_class_weights))

model_cw={}
model_cw['root']=root_cw_dict
model_cw['vowel']=vowel_cw_dict
model_cw['consonant']=cons_cw_dict

In [0]:
model =  build_se_resnext50_model_fs(height=HEIGHT, width=WIDTH, drop_out=DROP_OUT)

In [0]:
if 0: model.summary()

In [0]:
if 0:
  from tensorflow.keras.utils import plot_model
  plot_model(model, to_file=OUTPUT_DIR+'model3.png',show_shapes=True)

In [0]:
if 0:
  for (n, layer) in enumerate(model.layers[1].layers):
    if 'activation' in layer.get_config() and layer.get_config()['activation'] == 'relu':
      print('replacing #{}: {}, {}'.format(n, layer, layer.activation))
      layer.activation = Mish(mish)
      print('-> {}'.format(layer.activation))

In [0]:
TRAIN_DIR+PRETRAINED_WEIGHTS

'./train/'

In [0]:
if 0: 
  model.load_weights(TRAIN_DIR+PRETRAINED_WEIGHTS)
  valid_gen = test_batch_generator(valid_df,TRAIN_IMG_DIR,batch_size=512)
  root_predicts,vowel_predicts,cons_predicts=get_p_dicts(model,valid_gen)
  root_recalls,_,_ = compute_cm(valid_df,root_predicts,vowel_predicts,cons_predicts)
  root_weights = 1./np.power(root_recalls,2)
else:
  root_weights = 1.0


In [0]:
if 0: root_weights[62]

In [0]:
cutout_datagen = ImageDataGenerator(rotation_range = 20,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   preprocessing_function=get_random_eraser(s_l=0.05,s_h=0.1,r_1=0.8,r_2=1.2,v_l=0,v_h=0))

mcm_datagen = ImageDataGenerator(rotation_range = 20,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2)

In [0]:
columns=["root_class","vowel_class", "cons_class"]

mcm_generator1 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")

mcm_generator2 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")


cutout_generator = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")

Found 190798 validated image filenames.
Found 190798 validated image filenames.
Found 190798 validated image filenames.


In [0]:
def cutout_batch_fs(batch, p=0.5, s_l=0.05,s_h=0.3,r_1=0.2,r_2=5.0, st=STATS_FS):
        x,y = prep_batch_fs(batch,st=st)
        p_1 = np.random.rand()

        if p_1 > p:
            return x,y

        bs,img_h, img_w, img_c = x.shape

        for i in range(bs):
          while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

          x[i,top:top + h, left:left + w, :] = 0

        return x,y

In [0]:
def ps_batch_fs(batch, n=4, p=0.5, s_l=0.05,s_h=0.15,r_1=0.8,r_2=1.2, st=STATS_FS):
        x,y = prep_batch_fs(batch,st=st)
        p_1 = np.random.rand()

        if p_1 > p:
            return x,y

        bs,img_h, img_w, img_c = x.shape

        for i in range(bs):
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / n*r))
            h = int(np.sqrt(s * r/n))
            left = np.random.randint(0, img_w-w-1,size=n)
            top =  np.random.randint(0, img_h-h-1,size=n)
            for j in range(n):
              x[i,top[j]:top[j] + h, left[j]:left[j] + w, :] = 0

        return x,y

In [0]:
#cutout+progressive+mixup
def cpsm_generator_fs(gen1,gen2, gen3,st=STATS_FS):
    while True:
        p = np.random.rand()
        #print(f'p={p}')
        if p>0.5:
            batch = next(gen3)
            q  = np.random.rand()
            #print(f'q={q}')
            if q>0.5:
              #print('cutout')
              x_res,y_res = cutout_batch_fs(batch,st=st)
            else:
              #print('ps')
              ps_n = np.random.randint(4,16)
              x_res,y_res = ps_batch_fs(batch,n=ps_n)
        else:
            #print('mixup')
            batch1 = next(gen1)
            batch2 = next(gen2)
            mx_alpha = np.random.uniform(0.1,0.4)
            x_res,y_res = mix_batches_fs(batch1,batch2,alpha=mx_alpha,st=st)

        yield x_res,y_res

In [0]:
if 0:
  mixup_datagen=mixup_data_gen(aux_data_gen(mcm_generator1),aux_data_gen(mcm_generator2))
  cutout_datagen=aux_data_gen(cutout_generator)
  h_datagen=hybrid_data_gen(mixup_datagen,cutout_datagen)
  h2_datagen=h_generator(mcm_generator1,mcm_generator2,cutout_generator)
  hmcm_datagen=hmcm_generator(mcm_generator1,mcm_generator2)
  g3_datagen = g3_generator(mcm_generator1,mcm_generator2,cutout_generator)
  g3_datagen_fs = g3_generator_fs(mcm_generator1,mcm_generator2,cutout_generator,h=HEIGHT,w=WIDTH, st=STATS_FS)

cpsm_datagen_fs = cpsm_generator_fs(mcm_generator1,mcm_generator2,cutout_generator, st=STATS_FS)

In [0]:
if 0: 
  batch_x,y = next(cpsm_datagen_fs)
  import matplotlib.pyplot as plt
  fig, axs = plt.subplots(1, 7, figsize=(30, 30))
  for i in range(7):
    axs[i].imshow(batch_x[i,:,:,0])
  plt.show()

In [0]:
if 0: opt = Lookahead(tf.keras.optimizers.SGD(learning_rate=LR_MAX, momentum=0.9,nesterov=True))

opt=tf.keras.optimizers.Adam(learning_rate=LR_MAX)

In [0]:
checkpoint =  tf.train.Checkpoint(latest_epoch=tf.Variable(0), optimizer=opt, model=model)
chkp_manager =  tf.train.CheckpointManager(checkpoint, CHKP_DIR, max_to_keep=3)

In [0]:
chkp_manager.latest_checkpoint

'/content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-30'

In [0]:
chkp_restore_status=checkpoint.restore(chkp_manager.latest_checkpoint)
if chkp_manager.latest_checkpoint:
  #chkp_restore_status.assert_consumed()
  print("Restored from {}".format(chkp_manager.latest_checkpoint))
  print(f"Lastest epoch {checkpoint.latest_epoch.numpy()}")
else:
  print("Initializing from scratch.")

Restored from /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-30
Lastest epoch 30


In [0]:
loss_dict={'root':    'categorical_crossentropy',
           'vowel':    'categorical_crossentropy',
           'consonant':'categorical_crossentropy'}
model.compile(optimizer=opt, loss=loss_dict, loss_weights=[2.0,1.0,1.0])

In [0]:
initial_epoch = checkpoint.latest_epoch.numpy()

print(f'epochs trained: {initial_epoch}. start training epoch {initial_epoch+1}...')
#ca_shed = CosineAnnealingScheduler(T_max=T_MAX, eta_max=LR_MAX, eta_min=LR_MIN,verbose=1, initial_epoch=initial_epoch-130)
rolrplt = tf.keras.callbacks.ReduceLROnPlateau(monitor='kaggle',mode='max', patience=5, verbose=1, factor=0.95, min_lr=LR_MIN)


suffix =EXPERIMENT+\
                '_eps'+str(EPOCHS)+\
                '_iniep'+str(initial_epoch)+\
                '_sz'+str(IMG_SIZE)+\
                '_bs'+str(BATCH_SIZE)+\
                '_do'+str(DROP_OUT)+\
                '_'+model.optimizer.get_config()['name']+\
                '_IS_fold'+str(FOLD)


kv = KaggleValidationFS(valid_df,
                      batch_size=512,
                      vals_per_epoch=1,
                      train_steps=train_m//BATCH_SIZE+1,
                      logfile=EXPERIMENT_DIR+'log_kr_'+suffix+'.csv',
                      initial_epoch=initial_epoch,
                      suffix = suffix,
                      train_img_dir=TRAIN_IMG_DIR,
                      output_dir=EXPERIMENT_DIR,
                      thresholds=[0.98,0.995,0.993],
                      chkp_manager=chkp_manager,
                      in_shape=[HEIGHT,WIDTH],
                      st=STATS_FS)


cbs = [kv,rolrplt]

history=model.fit(cpsm_datagen_fs,
                  epochs=EPOCHS,
                  steps_per_epoch=train_m//BATCH_SIZE+1,
                  callbacks=cbs,
                  class_weight=model_cw,
                  verbose=2)


epochs trained: 30. start training epoch 31...
Train for 1491 steps
Epoch 1/200

batch 1490: validation...



[0:26:15] - kaggle:0.9742 - root:0.9614 -vowel:0.9873 - cons: 0.9866

saved checkpoint for epoch 31: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-31

1491/1491 - 1579s - loss: 1.4415 - root_loss: 0.4685 - vowel_loss: 0.2672 - consonant_loss: 0.2372
Epoch 2/200

batch 1490: validation...



[0:53:43] - kaggle:0.9729 - root:0.9611 -vowel:0.9872 - cons: 0.9822

saved checkpoint for epoch 32: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-32

1491/1491 - 1648s - loss: 1.4802 - root_loss: 0.4795 - vowel_loss: 0.2747 - consonant_loss: 0.2466
Epoch 3/200

batch 1490: validation...



[1:19:47] - kaggle:0.9761 - root:0.9663 -vowel:0.9860 - cons: 0.9860

saved checkpoint for epoch 33: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-33

1491/1491 - 1564s - loss: 1.4677 - root_loss: 0.4723 - vowel_loss: 0.2751 - consonant_loss: 0.2481
Epoch 4/200

batch 1490: validation...



[1:45:21] - kaggle:0.9755 - root:0.9641 -vowel:0.9871 - cons: 0.9868

saved checkpoint for epoch 34: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-34

1491/1491 - 1533s - loss: 1.4117 - root_loss: 0.4600 - vowel_loss: 0.2616 - consonant_loss: 0.2301
Epoch 5/200

batch 1490: validation...


...


[9:05:04] - kaggle:0.9778 - root:0.9672 -vowel:0.9879 - cons: 0.9890

saved checkpoint for epoch 51: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-51

1491/1491 - 1530s - loss: 1.2536 - root_loss: 0.4014 - vowel_loss: 0.2373 - consonant_loss: 0.2135
Epoch 22/200

batch 1490: validation...



[9:30:53] - kaggle:0.9787 - root:0.9686 -vowel:0.9886 - cons: 0.9888

saved checkpoint for epoch 52: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-52

1491/1491 - 1549s - loss: 1.2971 - root_loss: 0.4131 - vowel_loss: 0.2473 - consonant_loss: 0.2235
Epoch 23/200

batch 1490: validation...



[9:56:47] - kaggle:0.9804 - root:0.9721 -vowel:0.9901 - cons: 0.9874

saved checkpoint for epoch 53: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-53

1491/1491 - 1554s - loss: 1.2953 - root_loss: 0.4139 - vowel_loss: 0.2466 - consonant_loss: 0.2209
Epoch 24/200

batch 1490: validation...



[10:22:33] - kaggle:0.9769 - root:0.9657 -vowel:0.9876 - cons: 0.9886

saved checkpoint for epoch 54: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-54

1491/1491 - 1546s - loss: 1.2656 - root_loss: 0.4056 - vowel_loss: 0.2405 - consonant_loss: 0.2140
Epoch 25/200

batch 1490: validation...



[10:48:33] - kaggle:0.9790 - root:0.9703 -vowel:0.9879 - cons: 0.9876

saved checkpoint for epoch 55: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_wod3/chkp/ckpt-55

1491/1491 - 1560s - loss: 1.2561 - root_loss: 0.4008 - vowel_loss: 0.2381 - consonant_loss: 0.2164
Epoch 26/200


In [0]:
model.optimizer.get_config()